# IMPORTAR LIBRERIAS

In [1]:
import mysql.connector as con
import pandas as pd
import pymysql # Para ver la versión
import sqlalchemy # Para ver la versión
from sqlalchemy import create_engine

In [2]:
print(f"Conector: {con.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"pymysql: {pymysql.__version__}")
print(f"sqlalchemy: {sqlalchemy.__version__}")

Conector: 9.1.0
Pandas: 2.2.3
pymysql: 1.4.6
sqlalchemy: 2.0.37


# PARTE 1: CREAR BASE DE DATOS Y CREAR TABLAS

Código SQL para la creacion de base de datos y tablas

In [3]:
sql_creacion = """ 
DROP DATABASE IF EXISTS supermercado;

CREATE DATABASE IF NOT EXISTS supermercado;

USE supermercado;

CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(100),
    direccion VARCHAR(255),
    ciudad VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(100),
    puesto VARCHAR(50),
    id_tienda INT NOT NULL,
    FOREIGN KEY (id_tienda) REFERENCES tiendas (id_tienda)
);

CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nombre_categoria VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS productos (
    id_producto INT AUTO_INCREMENT PRIMARY KEY,
    nombre_producto VARCHAR(100),
    precio DECIMAL(5, 2),
    stock INT,
    id_categoria INT NOT NULL,
    FOREIGN KEY (id_categoria) REFERENCES categorias (id_categoria)
);

CREATE TABLE IF NOT EXISTS clientes (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(255),
    codigo_postal INT(5)
);

CREATE TABLE IF NOT EXISTS ordenes (
    id_orden INT AUTO_INCREMENT PRIMARY KEY,
    id_cliente INT NOT NULL,
    id_empleado INT NOT NULL,
    fecha_orden DATETIME,
    metodo_pago ENUM('Tarjeta', 'Efectivo'),
    FOREIGN KEY (id_cliente) REFERENCES clientes (id_cliente),
    FOREIGN KEY (id_empleado) REFERENCES empleados (id_empleado)
);

CREATE TABLE IF NOT EXISTS detalle_orden (
    id_detalle INT AUTO_INCREMENT PRIMARY KEY,
    id_orden INT NOT NULL,
    id_producto INT NOT NULL,
    cantidad INT,
    precio_unitario DECIMAL(5, 2), -- mismo precio que en la tabla producto
    descuento INT,
    FOREIGN KEY (id_orden) REFERENCES ordenes (id_orden),
    FOREIGN KEY (id_producto) REFERENCES productos (id_producto)
);
"""

Conexión a MySQL y ejecución de codigo de creacion

In [4]:
connection = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin"
)

cursor = connection.cursor()
cursor.execute(sql_creacion)
cursor.close()
connection.close()